In [16]:
from sklearn.cluster import KMeans
import pandas as pd, numpy as np
from matplotlib import pyplot as plt
from collections import Counter
from sklearn.metrics import silhouette_score
import math

from keras import Model
from keras.layers import LSTM, Input, Dense, Concatenate, Embedding, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
print(tf.config.list_physical_devices('GPU'))


[]


## Constants


In [17]:
batch_size = 100
debug =0

In [18]:
import pickle

f = open("./401_data/ohe_401", "rb")
ohe_401 =  pickle.load(f)

ohe_401.handle_unknown="ignore"
f.close()

f = open("./429_data/ohe_429", "rb")
ohe_429 =  pickle.load(f)
ohe_429.handle_unknown="ignore"
f.close()

In [19]:

# train=600000
df_for_training = pd.read_csv('delta_augumented_file.csv', nrows=2000000)#reading only PC and Delta
cols = df_for_training.columns
df_for_training = df_for_training[cols].astype(float)
cols = df_for_training.columns



df0 = df_for_training[df_for_training['cpu'] == 0]

df1 = df_for_training[df_for_training['cpu'] == 1]

dfs = [df0, df1]
print(df_for_training.shape)

oe_PCs = []
oe_Deltas = []

for i in range(2):

    MAX_DELTA = int(df0['Delta'].nunique()) + 1
    MAX_PC = int(df_for_training['PC'].nunique()) + 1


    oe_PC = OrdinalEncoder()
    oe_Delta = OrdinalEncoder()

    oe_PC = oe_PC.fit(dfs[i]['PC'].values.reshape(-1, 1))
    oe_Delta = oe_Delta.fit(dfs[i]['Delta'].values.reshape(-1, 1))

    oe_PCs.append(oe_PC)
    oe_Deltas.append(oe_Delta)



(2000000, 4)


In [20]:
model0 = load_model("./401_data/best_model_401.h5")
model1 = load_model("./429_data/best_model_429.h5")

/home/me/anaconda3/envs/menv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/me/anaconda3/envs/menv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [21]:



models =  [model0, model1]
df  = pd.read_csv("delta_augumented_file.csv", nrows=4000000)
#we have PC,Cache,Delta,CPU
hit_0 = 0
hit_1 = 0


q0 = []
q1 = []

oe_PC_401 = oe_PCs[0]
oe_PC_429 = oe_PCs[1]

oe_Delta_401 = oe_Deltas[0]
oe_Delta_429 = oe_Deltas[1]

old_delta0 = ""
old_delta1 = ""
##
#for handling unknown delta in ordinal encoder we randomly map them to any known delta encoding.
##

def process_vals(vals):
    pc = vals[0]
    cache = vals[1]
    delta  = vals[2]
    if(vals[-1] == 0):
        pc = oe_PC_401.transform([[pc]])
        delta = oe_Delta_401.transform([[delta]])

        q0.append((pc[0][0], delta[0][0]))
        # print(q0[-1])
        do_predict(0)
    else:
        pc = oe_PC_429.transform([[pc]])
        delta = oe_Delta_429.transform([[delta]])
        q1.append((pc[0][0], delta[0][0]))

        do_predict(1)


def do_predict(cpu):
    global q0, q1, hit_0, hit_1
    if(cpu == 0):
        if(len(q0) == 21):
            xs = np.array(q0[0:20])
            y = np.array([[q0[20:21][0][1]]]) 

            pcs = [a[0] for a in xs]
            deltas = [a[1] for a in xs]

            od = oe_Delta_401.inverse_transform(y)
            y_true = ohe_401.transform(od).argmax()


            print(pcs, deltas)

            y_predicted = models[0].predict([[pcs], [deltas]])
            y_predicted = y_predicted.argmax()
            if(y_true == y_predicted):
                hit_0+=1
            q0 = q0[1:]
    else:
        if(len(q1) == 21):
            xs = np.array(q1[0:20])
            y = np.array([[q1[20:21][0][1]]])

            pcs = [a[0] for a in xs]
            deltas = [a[1] for a in xs]

            od = oe_Delta_429.inverse_transform(y)
            y_true = ohe_429.transform(od).argmax()

            y_predicted = models[1].predict([[pcs], [deltas]])
            y_predicted = y_predicted.argmax()
            if(y_true == y_predicted):
                hit_1+=1
            q1 = q1[1:]
            
f = open("delta_augumented_file.csv")
for line in f:
    if "PC" in line:
        continue
    vals = list(map(int, line.strip().split(",")))
    process_vals(vals)

[0.0, 1.0, 1.0, 36.0, 36.0, 36.0, 36.0, 237.0, 237.0, 238.0, 238.0, 238.0, 239.0, 240.0, 240.0, 245.0, 246.0, 241.0, 241.0, 244.0] [1849.0, 1132.0, 1599.0, 1483.0, 1462.0, 1609.0, 1472.0, 1487.0, 1761.0, 1212.0, 1487.0, 1458.0, 1472.0, 1471.0, 1481.0, 1463.0, 1470.0, 1473.0, 1476.0, 1467.0]


InvalidArgumentError:  indices[0,0] = 1849 is not in [0, 264)
	 [[node embedding_2_4/embedding_lookup (defined at /home/me/anaconda3/envs/menv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_19514]

Function call stack:
keras_scratch_graph


In [46]:
hit_1

43

In [3]:

# train=600000
df_for_training = pd.read_csv('result_ii.txt', usecols=[1, 2],nrows=10000)#reading only PC and Delta
cols = df_for_training.columns
df_for_training = df_for_training[cols].astype(float)
cols = df_for_training.columns

print(df_for_training.shape)

vocab_deltas = pd.read_csv("vocab_deltas.txt", dtype=np.int32,nrows=2000)
vocab_deltas = vocab_deltas['top_deltas'].tolist()

print("vocab_deltas is of type",type(vocab_deltas[0]))
print("Vocab size is: ", len(vocab_deltas))
vocab_size = len(vocab_deltas)

MAX_DELTA = int(df_for_training['Delta'].nunique()) + 1
MAX_PC = int(df_for_training['PC'].nunique()) + 1


oe_PC = OrdinalEncoder()
oe_Delta = OrdinalEncoder()

oe_PC = oe_PC.fit(df_for_training['PC'].values.reshape(-1, 1))
oe_Delta = oe_Delta.fit(df_for_training['Delta'].values.reshape(-1, 1))


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe.fit(np.array(vocab_deltas).reshape(-1,1))

(10000, 2)
vocab_deltas is of type <class 'int'>
Vocab size is:  2


OneHotEncoder(handle_unknown='ignore', sparse=False)

In [4]:
from tensorflow.python.keras.utils import data_utils


class CustomDataGen(data_utils.Sequence):
    
    def __init__(self, df, batch_size, ohe, oe_pc, oe_delta):
        self.df = df.copy()
        self.n_future = 1
        self.n_past = 20
        self.batch_size = batch_size
        self.n = len(self.df) - self.n_past  ##calculate it precisely
        self.n = math.ceil(self.n // self.batch_size)
        self.n -=1
        self.ohe = ohe
        self.ix = 0
        self.ohe = ohe
        self.tmp_y = self.df['Delta'].copy().to_numpy()
        self.oe_pc = oe_pc
        self.oe_delta = oe_delta

        self.df['PC'] = self.oe_pc.transform(self.df['PC'].values.reshape(-1, 1))
        self.df['Delta'] = self.oe_delta.transform(self.df['Delta'].values.reshape(-1, 1))
        # print(self.df.head())
        self.df = self.df.to_numpy(dtype=np.float64)
        self.iterator_index = 0
        # print(self.df[0])

    def __next__(self):
        data = self.__getitem__(self.ix)
        self.ix  += 1
        self.ix %= self.n
        return data
    
    def __iter__(self):
        while True:
            data= self.__getitem__(self.iterator_index)
            self.iterator_index += 1
            self.iterator_index %= self.n
            yield data
    def reset_iterator(self):
        self.iterator_index = 0
    
    def __getitem__(self, index):
        global debug
        """
        index is batch_index
        each batch will have batch_size elements
        """
        index %= self.n
        X = []
        Y = []
        start = index * self.batch_size
        for i in range(start, start+batch_size):
            X.append(self.df[i:i + self.n_past, :])
            Y.append(self.ohe.transform(np.array(self.tmp_y[i + self.n_past : i + self.n_past + 1]).
                                        reshape(-1,1)))
            
        X = np.array(X)
        Y = np.array(Y)
        shp = Y.shape
        Y = Y.reshape(shp[0], shp[2])
        
        
        tmp = np.split(X, 2, axis=2)
        tmp[0] = tmp[0].reshape((batch_size, 20))
        tmp[1] = tmp[1].reshape((batch_size, 20))
        if (debug == 1 and index == 0):
          print(tmp, y)
        return (tmp, Y)

    def getdatapoint(self, i):
        return self.df[i:i+self.n_past, :]
    
    def __len__(self):
        return self.n

In [5]:
print(df_for_training.shape)

(10000, 2)


In [6]:
print(type(df_for_training))
df_train, df_test = train_test_split(df_for_training, test_size = 0.25, shuffle=False)

print(type(df_train))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [7]:

# df_train, df_test = train_test_split(df_for_training, test_size = 0.25, shuffle=False)
# df_train, df_validate = train_test_split(df_train, test_size = 0.20, shuffle=False)

# # df_test['PC'] = oe_PC.transform(df_test['PC'].values.reshape(-1, 1))

df_all = CustomDataGen(df_for_training, batch_size, ohe, oe_PC, oe_Delta)
# data_gen_train = CustomDataGen(df_train, batch_size, ohe, oe_PC, oe_Delta)
# data_gen_validate = CustomDataGen(df_validate, batch_size, ohe, oe_PC, oe_Delta)
# data_gen_test = CustomDataGen(df_test, batch_size, ohe, oe_PC, oe_Delta)

In [12]:

saved_model = load_model('best_model_401.h5')
df_all.reset_iterator()
Y_predict = saved_model.predict(df_all,
                                 steps=df_all.__len__(),  verbose=1)
# data_gen_test.reset_iterator()
train_loss = saved_model.evaluate(df_all,
                                      steps=None,  verbose=1)
print(train_loss)
print(Y_predict.shape)
print(type(Y_predict))
print(Y_predict.dtype)

/home/me/anaconda3/envs/menv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


98/98 [==============================] - 3s 29ms/step
[0.008457683026790619, 0.9280612468719482]
(9800, 2)
<class 'numpy.ndarray'>
float32


In [9]:
sub_total = []
print( data_gen_test.n)
for j in range(0, data_gen_test.n):
    _, y = data_gen_test.__getitem__(j)
    sub_total.append(y)
print(type(sub_total[0]))
Y_true = np.concatenate(sub_total, axis = 0)
print(Y_true.shape)


NameError: name 'data_gen_test' is not defined

In [ ]:
print(Y_true)

In [ ]:


from sklearn.metrics import confusion_matrix, average_precision_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


    

Y_test_comp = Y_true.argmax(axis=1)
Y_predict_comp = Y_predict.argmax(axis=1)
Y_predict_comp = Y_predict_comp[0:len(Y_test_comp)]

arr = confusion_matrix(Y_test_comp, Y_predict_comp)
print('Accuracy: {:.2f}'.format(accuracy_score(Y_test_comp, Y_predict_comp)))
print('Weighted Precision: {:.2f}'.format(precision_score(Y_test_comp, Y_predict_comp, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(Y_test_comp, Y_predict_comp, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(Y_test_comp, Y_predict_comp, average='weighted')))
print("#############################################")



Accuracy: 0.07
Weighted Precision: 0.00
Weighted Recall: 0.07
Weighted F1-score: 0.01
#############################################


/home/vijay/anaconda3/envs/ayushenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
tmp_y = df_test['Delta'].copy()

df_test['PC'] = oe_PC.transform(df_test['PC'].values.reshape(-1, 1))
df_test['Delta'] = oe_Delta.transform(df_test['Delta'].values.reshape(-1, 1))

testX = []
testY = []
n_past =20
n_future = 1

for i in range(n_past, len(df_test) - n_future +1):
    testX.append(df_test.iloc[i - n_past:i, :])#include pc and delta
    testY.append(ohe.transform(np.array(tmp_y[i + n_future - 1:i + n_future]).reshape(-1, 1)))
testX = np.array(testX)
testY = np.array(testY)



<ipython-input-11-fc71fa543719>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['PC'] = oe_PC.transform(df_test['PC'].values.reshape(-1, 1))
<ipython-input-11-fc71fa543719>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Delta'] = oe_Delta.transform(df_test['Delta'].values.reshape(-1, 1))


In [ ]:
shp = testY.shape
testY = testY.reshape(shp[0], shp[2])

In [ ]:
saved_model = load_model('best_model.h5')
train_loss = saved_model.evaluate(np.split(testX, 2, axis=2), testY, verbose=1)
Y_predict = saved_model.predict(np.split(testX, 2, axis=2))
print(train_loss)

7812/7812 [==============================] - 123s 16ms/step - loss: 0.9413 - accuracy: 0.7287
